In [67]:
import pandas as pd

In [68]:
props = pd.read_csv('../groupby_prep/props_clean.csv', low_memory=0)
props.drop_duplicates(subset='PID', inplace=True)
props['Land Use Pct. Agri./Comm./Indust.'] = props['SIMPLIFIED_LU_AGRICULTURAL']+props['SIMPLIFIED_LU_COMMERCIAL']+props['SIMPLIFIED_LU_INDUSTRIAL']
props['Land Use Pct. Condo/Residential'] = props['SIMPLIFIED_LU_CONDO']+props['SIMPLIFIED_LU_CONDO_UNIT']+props['SIMPLIFIED_LU_RESIDENTIAL']
props['Land Use Pct. Tax Exempt/TE-BPDA'] = props['SIMPLIFIED_LU_TAX_EXEMPT']+props['SIMPLIFIED_LU_TAX_EXEMPT_BRA']
props['Land Use Pct. Comm+Resid. Mixed'] = +props['SIMPLIFIED_LU_MIX_RC']

def props_groupby(group_area): 
    out = props.groupby(group_area).count()[['PID']]
    out = out.rename(columns={'PID':'Count Parcels'})
# groupby sum:
    for gb in ['unit_N', 'LIVING_AREA_org']: 
        out = pd.concat([out, props.groupby(group_area).sum()[gb]], axis=1, sort=False)
    out = out.rename(columns={'unit_N':'Number Units', 'LIVING_AREA_org':'total_living_area',})
# calc avg liv area / unit
    out['Avg Living Area / Unit'] = round(out['total_living_area'] / out['Count Units'])
    out.drop(columns=['total_living_area'], inplace=True) 
# groupby mean:
    landuse_dummies = [c for c in props.columns if 'Perc' in c]
    for gb in ['BLDG_AGE', 'OWN_OCC'] + landuse_dummies:
        out = pd.concat([out, props.groupby(group_area).mean()[gb]], axis=1, sort=False)
    out = out.rename(columns={'BLDG_AGE':'Avg Building Age', 'OWN_OCC':'Pct. units owner-occupied' })
    out = out.rename(columns={c: c.replace('landuse_','') for c in landuse_dummies})
    out.reset_index(inplace=True)
    out.rename(columns={'CT_name10': 'Census Tract (2010)',
                         'neighborhood': 'Neighborhood'}, inplace=True)
    out.rename(columns={c:'pro_'+c for c in out.columns}, inplace=True)
    return out

In [69]:
stre = pd.read_csv('../groupby_prep/stre_clean.csv', low_memory=0)

def stre_groupby(group_area):
    sum_groupby = [c for c in stre.columns if 'APP_Stat' in c] + ['PSD_open violation count',
                   'PSD_violations in the last 6 months']
    out = pd.DataFrame()
    for gb in sum_groupby:
        out = pd.concat([out, stre.groupby(group_area).sum()[gb]], axis=1, sort=False)
    out.index.name = group_area
    out.rename(columns={'PSD_violations in the last 6 months':
                        'Count violations/past 6 mos.'}, inplace=True)
    for c in out.columns:
        if 'APP' in c:
            out.rename(columns={c:': '.join(c.split('_')[-2:])+(' (count)') },
                       inplace=True)
    out.rename(columns={'PSD_open violation count':
                        'Count open violations'}, inplace=True)
    out.reset_index(inplace=True)
    out.rename(columns={'CT_name10': 'Census Tract (2010)',
                         'neighborhood': 'Neighborhood'}, inplace=True)
    out.rename(columns={c:'str_'+c for c in out.columns}, inplace=True)
    return out

In [70]:
airbnb = pd.read_csv('../groupby_prep/airbnb_clean.csv', low_memory=0)
def airbnb_groupby(group_area):
    out = pd.DataFrame()
    count_groupby = [ 'id' ]
    for gb in count_groupby:
        out = pd.concat([out, airbnb.groupby(['last_scraped', group_area]).count()[[gb]]
                        ], axis=1, sort=False)
        out = out.rename(columns={gb:'Count Listings'})
    sum_groupby = ['number_of_reviews', 'accommodates', 'has_license', 'exempt_license']
    for gb in sum_groupby:
        out = pd.concat([out, airbnb.groupby(['last_scraped', group_area]).sum()[[gb]]
                        ], axis=1, sort=False)
    out = out.rename(columns={'has_license': 'Count Licensed',
                              'exempt_license': 'Count Exempt',
                              'number_of_reviews': 'Count Reviews',
                              'accommodates':'Sum Accomodations'})
    out['Pct. Licensed'] = out['Count Licensed'] / out['Count Listings']
    out['Pct. License Exempt'] = out['Count Exempt'] / out['Count Listings']
    out['Avg. Reviews/Listing'] = out['Count Reviews'] / out['Count Listings']
    out.drop(columns=['Count Reviews'], inplace=True)
    
    out = pd.concat([out, round(airbnb.groupby(['last_scraped', group_area]).nunique()['host_id'], 2)], axis=1, sort=False)
    out = out.rename(columns={'host_id':'Unique Hosts (count)'})
    
    
    median_groupby = ['host_distance_km', 'price' ]
    for gb in median_groupby:
        out = pd.concat([out, airbnb.groupby(['last_scraped', group_area]).median()[gb]], axis=1, sort=False)
    out = out.rename(columns={'host_distance_km':'Med. Host Distance (km.)'})
    out = out.rename(columns={'price':'Med. Price'})

    
    mean_groupby = ['price', 'availability_30']
    for gb in mean_groupby:
        out = pd.concat([out, round(airbnb.groupby(['last_scraped', group_area]).mean()[gb], 2)], axis=1, sort=False)
    out = out.rename(columns={'price':'Avg. Price'})
    out.reset_index(inplace=True)
    out.rename(columns={'availability_30':'Avg. Avlblty / Next 30d',
                        'last_scraped':'scraped'}, inplace=True)
    out.scraped = pd.to_datetime(out.scraped)
    out.rename(columns={'CT_name10': 'Census Tract (2010)',
                         'neighborhood': 'Neighborhood'}, inplace=True)
    out.rename(columns={c:'air_'+c for c in out.columns}, inplace=True)
    return out

In [71]:
padm = pd.read_csv('../groupby_prep/padmapper_clean.csv', low_memory=0)
def padm_groupby(group_area):
    out = pd.DataFrame()
    out = pd.concat([out,
                     padm.groupby(['scraped', group_area]).nunique()[
                         ['id']
                     ]], axis=1, sort=False)
    out = out.rename(columns={'id':'Count Listings'})
    out = pd.concat([out,
                     padm.groupby(['scraped', group_area]).median()[
                         ['price']
                     ]], axis=1, sort=False)
    out = out.rename(columns={'price':'Med. Price/mo.'})
    out = pd.concat([out,
                     padm.groupby(['scraped', group_area]).mean()[
                         ['price', 'beds']
                     ]], axis=1, sort=False)
    out = out.rename(columns={'price':'Avg. Price/mo.', 'beds':'Avg. Beds'})
    out['Med. Price/mo./Bed'] = out['Med. Price/mo.']/out['Avg. Beds']
    out['Avg. Price/mo./Bed'] = out['Avg. Price/mo.']/out['Avg. Beds']
    out.reset_index(inplace=True)
    out.scraped = pd.to_datetime(out.scraped)
    out.rename(columns={'CT_name10': 'Census Tract (2010)',
                         'neighborhood': 'Neighborhood'}, inplace=True)
    out.rename(columns={c:'pad_'+c for c in out.columns}, inplace=True)
    return out

# do it

wait a bit between cells to avoid crashing.

In [78]:
group_area = 'neighborhood'

In [79]:
bari_data = props_groupby(group_area)

In [80]:
air_data = airbnb_groupby(group_area)

In [81]:
str_data = stre_groupby(group_area) 

In [82]:
pad_data = padm_groupby(group_area)

In [83]:
if 'C' in group_area: path='c_grouped'
if 'g' in group_area: path='n_grouped'

for name, data in {'air': air_data,
                   'str':str_data,
                   'pro':bari_data,
                   'pad':pad_data}.items():
    for c in data.columns:
        if 'Pct' in c: data[c] = round(data[c]*100, 2)
    data = round(data, 2)
    data.to_csv(f'{path}/{name}.csv', index=False)

In [28]:
features_dict = {'air_Count Listings': 'Total number of unique Airbnb listings',
 'air_Sum Accodomations': 'Sum total of accomodations (guests) of Airbnb listings',
 'air_Count Licensed': 'Total number of Airbnb listings with an STR license claimed on airbnb.com',
 'air_Count Exempt': 'Total number of Airbnb listings with an STR Exemption claimed on airbnb.com',
 'air_Perc. Licensed':  'Percentage of Airbnb listings with an STR License',
 'air_Perc. License Exempt':  'Percentage of Airbnb listings with an STR License exemption',
 'air_Avg. Reviews/Listing':  'Average number of reviews (all-time) per Airbnb listing',
 'air_Unique Hosts (count)': 'Total number of Airbnb hosts',
 'air_Med. Host Distance (km.)': 'Median distance between Airbnb listings and their hosts (approximated within 5km)',
 'air_Med. Price': 'Median price per night of Airbnb listings ($)',
 'air_Avg. Price': 'Average price per night of Airbnb listings ($)',
 'air_Avg. Avlblty / Next 30d': 'Average availability (number of days not booked) per Airbnb listing over the next 30 days',
 
 'pad_Med. Price/mo.': 'Median price per month of rental listings',
 'pad_Avg. Price/mo.': 'Average price per month of rental listings',
 'pad_Avg. Beds': 'Average number of bedrooms per listing',
 'pad_Med. Price/mo./Bed': 'Median price per month per bedroom',
 'pad_Avg. Price/mo./Bed': 'Average price per month per bedroom',
 'pro_Count Parcels': 'Total number of unique Parcel Ids or properties in an area',
 'pro_Count Units':  'Total number of units in (all zoning types)',
 'pro_Avg Living Area / Unit': 'Average living area per unit (total number of units divided by total Living Area)',
 'pro_Avg Building Age': 'Average age of buildings (years)',
 'pro_Perc. units owner-occupied': 'Percentage of units that are owner-occupied',
 'pro_LandUsePerc. Agri./Comm./Indust.': 'Percentage of units zoned Agricultural, Commercial or Industrial',
 'pro_LandUsePerc. Condo/Res./C.R.Mix': 'Percentage of units zoned Condo, Complex, Residential or C/R Mixed',
 'pro_LandUsePerc. Tax Exempt/TE-BPDA': 'Percentage of units zoned Tax-Exempt or Tax-Exempt through the BPDA',
 'str_Status: Active (count)': 'Total number of Active STR-registered units',
 'str_Status: Expired (count)': 'Total number of Expired STR registrations',
 'str_Status: Inactive (count)': 'Total number of Inactive STR registrations',
 'str_Status: Revoked (count)': 'Total number of Revoked STR registrations',
 'str_Status: Void (count)': 'Total number of Voided STR registrations',
 'str_Count open violations': 'Total count of open violations. From data.boston.gov: "Violations counted include: violations of the sanitary, building, zoning, and fire code; stop work orders; and abatement orders."',
 'str_Count violations/past 6 mos.': 'Total count of violations in the past 6 months. From data.boston.gov: "Violations counted include: violations of the sanitary, building, zoning, and fire code; stop work orders; and abatement orders."',
}